In [1]:
import pandas as pd
import numpy as np
from pandas.io import sql
from sqlalchemy import create_engine
import csv
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from surprise import Reader, Dataset

tqdm.pandas()

/Users/max/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
db_user = 'postgres'
db_password = ''
db_host = 'localhost'
db_port = 5432
database = 'climbing'

conn_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{database}'
connection = create_engine(conn_str)

In [3]:
df_clusters = pd.read_sql('select * from clusters',connection,index_col='index')

In [4]:
cluster_dict = df_clusters.T.to_dict()
rating = []
for user in tqdm(cluster_dict):
    for thing in cluster_dict[user]:
        rating.append([user,thing,cluster_dict[user][thing]])

In [5]:
df_rating = pd.DataFrame(rating,columns=['user','cluster','rat'])

In [6]:
df_rating

,user,cluster,rat
0,100002,cluster_0,0.000000
1,100002,cluster_1,0.000000
2,100002,cluster_2,0.000000
3,100002,cluster_3,0.000000
4,100002,cluster_4,0.000000
...,...,...,...
1322475,99999,cluster_35,0.000000
1322476,99999,cluster_36,0.000000
1322477,99999,cluster_37,0.092308
1322478,99999,cluster_38,0.000000


In [7]:
reader= Reader(rating_scale=(0,1))

data = Dataset.load_from_df(df_rating,reader)

In [8]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25)

In [9]:
import sklearn

In [10]:
from surprise import SVD, accuracy,SlopeOne
from surprise.model_selection.search import GridSearchCV

In [11]:
for i in range(1,50,10):
    print(i)

1
11
21
31
41


In [12]:
#try slopeone, cosine similarities
# grid_params = {'n_epochs':range(1,50,10)}

algo = SVD()
# gscv = GridSearchCV(SVD,grid_params,cv=5,n_jobs=-1,joblib_verbose=1)
algo.fit(trainset)
predicitons = algo.test(testset)

In [13]:
from surprise import accuracy

In [14]:
accuracy.rmse(predicitons)

RMSE: 0.0551


0.055087812134741905

In [15]:
from surprise.model_selection import cross_validate

In [16]:
cross_validate(algo, data, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0560  0.0563  0.0541  0.0552  0.0553  0.0554  0.0008  
MAE (testset)     0.0153  0.0157  0.0154  0.0155  0.0156  0.0155  0.0001  
Fit time          52.67   53.67   57.13   55.22   54.92   54.72   1.51    
Test time         1.76    2.87    3.25    1.67    3.81    2.67    0.84    


{'test_rmse': array([0.05604606, 0.05629977, 0.05413847, 0.05524828, 0.0553402 ]),
 'test_mae': array([0.01528279, 0.015655  , 0.015408  , 0.01549958, 0.01563877]),
 'fit_time': (52.66636800765991,
  53.6676459312439,
  57.13153600692749,
  55.221237897872925,
  54.91775393486023),
 'test_time': (1.7640609741210938,
  2.8695619106292725,
  3.248008966445923,
  1.6650030612945557,
  3.8054280281066895)}

In [17]:
algo.test([df_rating.loc[279250]])

[Prediction(uid=14570, iid='cluster_10', r_ui=0.0, est=0.0033140286463013147, details={'was_impossible': False})]

In [20]:
# from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
# from surprise.model_selection import cross_validate

# benchmark = []
# # Iterate over all algorithms
# for algorithm in tqdm([KNNWithMeans(),SVD(), NormalPredictor(),BaselineOnly()]):
#     # Perform cross validation
#     print(algorithm)
#     try:
#         results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
#     # Get results & append algorithm name
#         tmp = pd.DataFrame.from_dict(results).mean(axis=0)
#         tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
#         benchmark.append(tmp)
#     except:
#         print('failed')
    
# pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')   

In [21]:
df_clusters.columns

Index(['cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4',
       'cluster_5', 'cluster_6', 'cluster_7', 'cluster_8', 'cluster_9',
       'cluster_10', 'cluster_11', 'cluster_12', 'cluster_13', 'cluster_14',
       'cluster_15', 'cluster_16', 'cluster_17', 'cluster_18', 'cluster_19',
       'cluster_20', 'cluster_21', 'cluster_22', 'cluster_23', 'cluster_24',
       'cluster_25', 'cluster_26', 'cluster_27', 'cluster_28', 'cluster_29',
       'cluster_30', 'cluster_31', 'cluster_32', 'cluster_33', 'cluster_34',
       'cluster_35', 'cluster_36', 'cluster_37', 'cluster_38', 'cluster_39'],
      dtype='object')

In [22]:
df_clusters.loc[221061]

cluster_0     0.000000
cluster_1     0.103448
cluster_2     0.103448
cluster_3     0.000000
cluster_4     0.000000
cluster_5     0.000000
cluster_6     0.000000
cluster_7     0.000000
cluster_8     0.000000
cluster_9     0.000000
cluster_10    0.068966
cluster_11    0.000000
cluster_12    0.034483
cluster_13    0.000000
cluster_14    0.000000
cluster_15    0.000000
cluster_16    0.103448
cluster_17    0.000000
cluster_18    0.000000
cluster_19    0.034483
cluster_20    0.103448
cluster_21    0.000000
cluster_22    0.000000
cluster_23    0.068966
cluster_24    0.034483
cluster_25    0.000000
cluster_26    0.000000
cluster_27    0.000000
cluster_28    0.000000
cluster_29    0.000000
cluster_30    0.000000
cluster_31    0.034483
cluster_32    0.000000
cluster_33    0.000000
cluster_34    0.034483
cluster_35    0.000000
cluster_36    0.034483
cluster_37    0.000000
cluster_38    0.000000
cluster_39    0.241379
Name: 221061, dtype: float64

In [23]:
predictions_user = []
for column in df_clusters.columns:
    print(column)
    predictions_user.append((column,algo.predict(221061,column).est))

cluster_0
cluster_1
cluster_2
cluster_3
cluster_4
cluster_5
cluster_6
cluster_7
cluster_8
cluster_9
cluster_10
cluster_11
cluster_12
cluster_13
cluster_14
cluster_15
cluster_16
cluster_17
cluster_18
cluster_19
cluster_20
cluster_21
cluster_22
cluster_23
cluster_24
cluster_25
cluster_26
cluster_27
cluster_28
cluster_29
cluster_30
cluster_31
cluster_32
cluster_33
cluster_34
cluster_35
cluster_36
cluster_37
cluster_38
cluster_39


In [30]:
df_clusters.loc[221061]

cluster_0     0.000000
cluster_1     0.103448
cluster_2     0.103448
cluster_3     0.000000
cluster_4     0.000000
cluster_5     0.000000
cluster_6     0.000000
cluster_7     0.000000
cluster_8     0.000000
cluster_9     0.000000
cluster_10    0.068966
cluster_11    0.000000
cluster_12    0.034483
cluster_13    0.000000
cluster_14    0.000000
cluster_15    0.000000
cluster_16    0.103448
cluster_17    0.000000
cluster_18    0.000000
cluster_19    0.034483
cluster_20    0.103448
cluster_21    0.000000
cluster_22    0.000000
cluster_23    0.068966
cluster_24    0.034483
cluster_25    0.000000
cluster_26    0.000000
cluster_27    0.000000
cluster_28    0.000000
cluster_29    0.000000
cluster_30    0.000000
cluster_31    0.034483
cluster_32    0.000000
cluster_33    0.000000
cluster_34    0.034483
cluster_35    0.000000
cluster_36    0.034483
cluster_37    0.000000
cluster_38    0.000000
cluster_39    0.241379
Name: 221061, dtype: float64

In [54]:
df_clusters.loc[221061].values

array([0.13793103, 0.        , 0.03448276, 0.03448276, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.03448276,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.03448276, 0.06896552, 0.06896552, 0.03448276, 0.        ,
       0.03448276, 0.        , 0.        , 0.03448276, 0.        ,
       0.06896552, 0.03448276, 0.        , 0.06896552, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.03448276, 0.        , 0.03448276, 0.        , 0.24137931])

In [59]:
df_results = pd.DataFrame(predictions_user)

In [60]:
df_results['real'] = df_clusters.loc[221061].values

In [63]:
df_results.sort_values(by=1)

,0,1,real
18,cluster_18,0.013179,0.034483
25,cluster_25,0.013502,0.068966
15,cluster_15,0.013581,0.034483
27,cluster_27,0.013631,0.000000
28,cluster_28,0.013810,0.068966
7,cluster_7,0.013862,0.000000
38,cluster_38,0.014006,0.000000
17,cluster_17,0.014244,0.068966
4,cluster_4,0.014335,0.000000
33,cluster_33,0.014376,0.000000
